In [0]:
files =[f.name for f in  dbutils.fs.ls('/Volumes/workspace/default/databrics_data/')]
print(files)

In [0]:
files = ['Cola.xlsx', 'CompanyData.txt', 'etrm_trades.csv', 'iris.json', 'yiedl_daily.parquet']

csv_files = [f for f in files if f.endswith('.csv')]

print("CSV files only:", csv_files)
